In [1]:
%load_ext autoreload
%autoreload 2
from config import EXTRACTED_KEYWORDS_PATH, KEYWORDS_FINAL_PATH, CLUSTERS_FINAL_PATH, GRANTS_FILE
import pandas as pd
from visualisation import create_research_landscape_treemap, load_data

In [2]:
cat, classifcation = load_data()

Found 10 grant classifications
Found 5 categories


In [4]:
create_research_landscape_treemap()

Found 10 grant classifications
Found 5 categories
Total keywords: 101
Total grants: 10
Classified grants: 37
Average keywords per category: 20.2

Treemap contains 111 total elements:
  - 5 FOR Classes (Blue)
  - 5 Categories (Orange)
  - 101 Keywords (Green)

FOR Class distribution by keyword count:
  - FOR 40: ENGINEERING: 29 keywords
  - FOR 44: HUMAN SOCIETY: 29 keywords
  - FOR 41: ENVIRONMENTAL SCIENCES: 23 keywords
  - FOR 31: BIOLOGICAL SCIENCES: 10 keywords
  - FOR 32: BIOMEDICAL AND CLINICAL SCIENCES: 10 keywords

Classification summary:
Classified grants: 37
  Grant distribution by category:
    - Climate Science, Forcing and Long-term Variability: 2 grants
    - Geochemical Fingerprinting and Isotope Techniques: 2 grants
    - Corrosion, Surface Functionalization and Coatings: 1 grants
    - Catalysis and Green Chemical Processes: 1 grants
    - Mental Health, Neurobiology and Ageing: 1 grants
